In [1]:
import sys
from pathlib import Path
import pandas as pd
import unicodedata
import re
from io import StringIO
from typing import Iterable, Optional, Union


# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent.parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import INGRESOS_SALIDA_PATH,ARCHIVO_INGRESOS_FINAL_PATH, EGRESOS_SALIDA_PATH, ARCHIVO_EGRESOS_FINAL_PATH

In [2]:
# ---------- utilidades ----------
def ascii_key(s: str) -> str:
    """Llave simplificada para comparar encabezados: sin acentos ni símbolos."""
    s = str(s)
    s = s.replace("\ufeff","").replace("ï»¿","")
    # reparar secuencias comunes
    rep = {"Ã¡":"a","Ã©":"e","Ãí":"i","Ã³":"o","Ãº":"u","Ãñ":"n",
           "Â":"", "¿":"", "¡":"", "º":"", "°":"", "i>":""}
    for k,v in rep.items():
        s = s.replace(k,v)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    # quitar todo lo que no sea letra/numero/espacio
    s = re.sub(r"[^a-z0-9 ]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def leer_csv_robusto(path_csv: Path) -> pd.DataFrame:
    """Prueba encodings comunes y, si fallan, re-decodifica como latin1 y corrige."""
    for enc in ("utf-8-sig","cp1252","latin1"):
        try:
            df = pd.read_csv(path_csv, sep=";", encoding=enc, low_memory=False)
            if any("Ã" in c or "ï»¿" in c or "¿" in c for c in map(str, df.columns)):
                raise UnicodeDecodeError(enc,b"",0,1,"garbled")
            return df
        except Exception:
            pass
    raw = path_csv.read_bytes()
    text = raw.decode("latin1", errors="replace").replace("\ufeff","").replace("ï»¿","")
    return pd.read_csv(StringIO(text), sep=";", low_memory=False)

# ---------- pipeline ----------
def limpiar_y_agregar_fecha_denominacion(entrada: Path, salida: Path) -> pd.DataFrame:
    df = leer_csv_robusto(entrada)

    # mapa de llaves -> columna real
    keys = {ascii_key(c): c for c in df.columns}

    # --- detectar FECHA DE EMISIÓN ---
    col_fecha = None
    for k, c in keys.items():
        if "fecha" in k and ("emisi" in k or "emision" in k):
            col_fecha = c
            break
        # fallback: muchas veces queda "fecha de emisi n" (sin ó)
        if re.search(r"\bfecha\b.*\bemisi", k):
            col_fecha = c
            break
    if col_fecha is not None:
        df.rename(columns={col_fecha: "FechaEmision"}, inplace=True)
        df["FechaEmision"] = pd.to_datetime(df["FechaEmision"], errors="coerce")
    else:
        print("⚠️ No pude detectar la columna de fecha. La creo vacía.")
        df["FechaEmision"] = pd.NaT

    # --- detectar DENOMINACIÓN RECEPTOR ---
    col_denom = None
    for k, c in keys.items():
        if "denomin" in k and "receptor" in k:
            col_denom = c; break
    if col_denom is not None:
        df.rename(columns={col_denom: "DenominacionReceptor"}, inplace=True)
        df["DenominacionReceptor"] = df["DenominacionReceptor"].astype(str).fillna("Sin cliente")
    else:
        print("⚠️ No pude detectar 'Denominación Receptor'. La creo vacía.")
        df["DenominacionReceptor"] = "Sin cliente"

    # --- agregar AÑO y MES a partir de FechaEmision ---
    df["Año"] = df["FechaEmision"].dt.year
    df["Mes"] = df["FechaEmision"].dt.month

    # --- convertir importes a enteros (si existen) ---
    cols_numericas = [
        "Imp. Neto Gravado IVA 0%",
        "IVA 2,5%", "Imp. Neto Gravado IVA 2,5%",
        "IVA 5%", "Imp. Neto Gravado IVA 5%",
        "IVA 10,5%", "Imp. Neto Gravado IVA 10,5%",
        "IVA 21%", "Imp. Neto Gravado IVA 21%",
        "IVA 27%", "Imp. Neto Gravado IVA 27%",
        "Imp. Neto Gravado Total",
        "Imp. Neto No Gravado",
        "Imp. Op. Exentas",
        "Otros Tributos",
        "Total IVA",
        "Imp. Total",
    ]
    for col in cols_numericas:
        if col in df.columns:
            df[col] = (
                df[col].astype(str)
                      .str.replace(",", ".", regex=False)
                      .str.replace(r"[^0-9\.-]", "", regex=True)
                      .replace("", "0")
                      .astype(float).fillna(0).astype(int)
            )

    # --- TotalVenta e inversión de notas de crédito ---
    if "Imp. Total" in df.columns:
        df["TotalVenta"] = df["Imp. Total"].astype(int)
    if "Tipo de Comprobante" in df.columns:
        df["Tipo de Comprobante"] = df["Tipo de Comprobante"].astype(str)
        df.loc[df["Tipo de Comprobante"].isin(["3","8","203"]), "TotalVenta"] *= -1

    # --- columnas de salida: lo que pediste + Año/Mes/TotalVenta (útiles) ---
    cols_finales = [
        "FechaEmision", "Año", "Mes",
        "Tipo de Comprobante", "Nro. Doc. Receptor", "DenominacionReceptor",
        "Imp. Neto Gravado IVA 0%",
        "IVA 2,5%", "Imp. Neto Gravado IVA 2,5%",
        "IVA 5%", "Imp. Neto Gravado IVA 5%",
        "IVA 10,5%", "Imp. Neto Gravado IVA 10,5%",
        "IVA 21%", "Imp. Neto Gravado IVA 21%",
        "IVA 27%", "Imp. Neto Gravado IVA 27%",
        "Imp. Neto Gravado Total", "Imp. Neto No Gravado",
        "Imp. Op. Exentas", "Otros Tributos", "Total IVA","Imp. Total",
        "TotalVenta",
    ]
    cols_finales = [c for c in cols_finales if c in df.columns]
    out = df[cols_finales].copy()

    # guardar
    salida.parent.mkdir(parents=True, exist_ok=True)
    out.to_csv(salida, index=False, sep=";", encoding="utf-8-sig")

    print("✅ Listo. Exportado:", salida)
    print("📊 Filas:", len(out), "| Columnas:", len(out.columns))
    return out

In [3]:
# --- ejecutar ---
df_final = limpiar_y_agregar_fecha_denominacion(INGRESOS_SALIDA_PATH,ARCHIVO_INGRESOS_FINAL_PATH)
df_final.head()

✅ Listo. Exportado: ..\..\3_data_final\ingresos_totales_final.csv
📊 Filas: 126706 | Columnas: 24


,FechaEmision,Año,Mes,Tipo de Comprobante,Nro. Doc. Receptor,DenominacionReceptor,Imp. Neto Gravado IVA 0%,"IVA 2,5%","Imp. Neto Gravado IVA 2,5%",IVA 5%,...,Imp. Neto Gravado IVA 21%,IVA 27%,Imp. Neto Gravado IVA 27%,Imp. Neto Gravado Total,Imp. Neto No Gravado,Imp. Op. Exentas,Otros Tributos,Total IVA,Imp. Total,TotalVenta
0,2023-01-02,2023,1,1,20083459728,SUCESION DE CROCCE ARMANDO HECTOR,0,0,0,0,...,2066,0,0,2066,0,0,0,433,2500,2500
1,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,1636,0,0,1636,0,0,0,343,1980,1980
2,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,743,0,0,743,0,0,0,156,900,900
3,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,1322,0,0,1322,0,0,0,277,1600,1600
4,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,1157,0,0,1157,0,0,0,242,1400,1400


In [5]:
# --- ejecutar ---
df_final = limpiar_y_agregar_fecha_denominacion(EGRESOS_SALIDA_PATH,ARCHIVO_EGRESOS_FINAL_PATH)
df_final.head()

⚠️ No pude detectar 'Denominación Receptor'. La creo vacía.
✅ Listo. Exportado: ..\..\3_data_final\egresos_totales_final.csv
📊 Filas: 7852 | Columnas: 24


,FechaEmision,Año,Mes,Tipo de Comprobante,Nro. Doc. Receptor,DenominacionReceptor,Imp. Neto Gravado IVA 0%,"IVA 2,5%","Imp. Neto Gravado IVA 2,5%",IVA 5%,...,Imp. Neto Gravado IVA 21%,IVA 27%,Imp. Neto Gravado IVA 27%,Imp. Neto Gravado Total,Imp. Neto No Gravado,Imp. Op. Exentas,Otros Tributos,Total IVA,Imp. Total,TotalVenta
0,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,0,0,...,8115,0,0,8115,0,0,0,1704,9820,9820
1,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,0,0,...,29121,0,0,29121,0,0,0,6115,36402,36402
2,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,0,0,...,5700,0,0,5700,0,0,0,1197,7801,7801
3,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,0,0,...,41157,0,0,41157,0,0,0,8642,49800,49800
4,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,0,0,...,0,0,0,480335,0,0,0,50435,549983,549983


In [7]:
def columnas_todo_cero(
    df: pd.DataFrame,
    excluir: Optional[Iterable[str]] = None,
    umbral_ceros: float = 1.0,
) -> list[str]:
    """
    Devuelve columnas a eliminar:
      - numéricas que son TODO ceros (o proporción de ceros >= umbral_ceros)
      - columnas con TODO NaN también se eliminan
    Parámetros:
      excluir: columnas que nunca deben eliminarse
      umbral_ceros: 1.0 = solo todo ceros; 0.99 = casi todo ceros
    """
    excluir = set(excluir or [])
    eliminar: list[str] = []

    for col in df.columns:
        if col in excluir:
            continue

        s = df[col]
        # Si ya es numérica, perfecto; si no, intentamos convertir
        if not pd.api.types.is_numeric_dtype(s):
            s_num = pd.to_numeric(s, errors="coerce")
        else:
            s_num = s

        # TODO NaN -> eliminar
        if s_num.notna().sum() == 0:
            eliminar.append(col)
            continue

        # proporción de ceros (ignora NaN)
        zeros_share = (s_num.fillna(0) == 0).mean()

        if zeros_share >= umbral_ceros:
            eliminar.append(col)

    return eliminar


def eliminar_columnas_cero_df(
    df: pd.DataFrame,
    excluir: Optional[Iterable[str]] = None,
    umbral_ceros: float = 1.0,
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Elimina del DataFrame las columnas devueltas por columnas_todo_cero().
    """
    a_eliminar = columnas_todo_cero(df, excluir=excluir, umbral_ceros=umbral_ceros)
    if verbose and a_eliminar:
        print(f"🧽 Columnas eliminadas (todo ceros / NaN, umbral={umbral_ceros}): {a_eliminar}")
    return df.drop(columns=a_eliminar, errors="ignore")


def eliminar_columnas_cero_archivo(
    archivo_entrada: Union[str, Path],
    archivo_salida: Optional[Union[str, Path]] = None,
    excluir: Optional[Iterable[str]] = None,
    umbral_ceros: float = 1.0,
    sep: str = ";",
    encoding_in: str = "utf-8-sig",
    encoding_out: str = "utf-8-sig",
    verbose: bool = True,
) -> Path:
    """
    Lee un CSV, elimina columnas con todo ceros/NaN (o casi todo ceros según umbral)
    y guarda el resultado.
      - Si archivo_salida es None, genera *_sin_ceros.csv junto al original.
      - 'excluir' protege columnas clave para que no se eliminen.
    """
    archivo_entrada = Path(archivo_entrada)
    if archivo_salida is None:
        archivo_salida = archivo_entrada.with_name(archivo_entrada.stem + "_sin_ceros.csv")
    archivo_salida = Path(archivo_salida)

    df = pd.read_csv(archivo_entrada, sep=sep, encoding=encoding_in, low_memory=False)
    cols_ini = df.shape[1]
    df2 = eliminar_columnas_cero_df(df, excluir=excluir, umbral_ceros=umbral_ceros, verbose=verbose)

    archivo_salida.parent.mkdir(parents=True, exist_ok=True)
    df2.to_csv(archivo_salida, index=False, sep=sep, encoding=encoding_out)

    if verbose:
        print(f"✅ Guardado: {archivo_salida}  (columnas: {cols_ini} ➜ {df2.shape[1]})")
    return archivo_salida

In [8]:
# columnas que NO querés eliminar nunca
proteger = [
    "FechaEmision", "Año", "Mes",
    "Tipo de Comprobante", "Nro. Doc. Receptor", "DenominacionReceptor",
]

# umbral: 1.0 = solo columnas todo ceros; 0.99 = columnas con ≥99% ceros
archivo_salida = eliminar_columnas_cero_archivo(
    archivo_entrada=ARCHIVO_INGRESOS_FINAL_PATH,
    archivo_salida=ARCHIVO_INGRESOS_FINAL_PATH,  # sobrescribe el final (o dejalo en None para *_sin_ceros.csv)
    excluir=proteger,
    umbral_ceros=1.0,
    sep=";",
    encoding_in="utf-8-sig",
    encoding_out="utf-8-sig",
    verbose=True,
)

# vista rápida
pd.read_csv(archivo_salida, sep=";", encoding="utf-8-sig").head()


🧽 Columnas eliminadas (todo ceros / NaN, umbral=1.0): ['Imp. Neto No Gravado', 'Otros Tributos']
✅ Guardado: ..\..\3_data_final\ingresos_totales_final.csv  (columnas: 24 ➜ 22)


,FechaEmision,Año,Mes,Tipo de Comprobante,Nro. Doc. Receptor,DenominacionReceptor,Imp. Neto Gravado IVA 0%,"IVA 2,5%","Imp. Neto Gravado IVA 2,5%",IVA 5%,...,"Imp. Neto Gravado IVA 10,5%",IVA 21%,Imp. Neto Gravado IVA 21%,IVA 27%,Imp. Neto Gravado IVA 27%,Imp. Neto Gravado Total,Imp. Op. Exentas,Total IVA,Imp. Total,TotalVenta
0,2023-01-02,2023,1,1,20083459728,SUCESION DE CROCCE ARMANDO HECTOR,0,0,0,0,...,0,433,2066,0,0,2066,0,433,2500,2500
1,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,0,343,1636,0,0,1636,0,343,1980,1980
2,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,0,156,743,0,0,743,0,156,900,900
3,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,0,277,1322,0,0,1322,0,277,1600,1600
4,2023-01-02,2023,1,6,20111111112,CUIT GENERICO,0,0,0,0,...,0,242,1157,0,0,1157,0,242,1400,1400


In [ ]:
# columnas que NO querés eliminar nunca
proteger = [
    "FechaEmision", "Año", "Mes",
    "Tipo de Comprobante", "Nro. Doc. Receptor", "DenominacionReceptor",
]

# umbral: 1.0 = solo columnas todo ceros; 0.99 = columnas con ≥99% ceros
archivo_salida = eliminar_columnas_cero_archivo(
    archivo_entrada=ARCHIVO_EGRESOS_FINAL_PATH,
    archivo_salida=ARCHIVO_EGRESOS_FINAL_PATH,  # sobrescribe el final (o dejalo en None para *_sin_ceros.csv)
    excluir=proteger,
    umbral_ceros=1.0,
    sep=";",
    encoding_in="utf-8-sig",
    encoding_out="utf-8-sig",
    verbose=True,
)

# vista rápida
pd.read_csv(archivo_salida, sep=";", encoding="utf-8-sig").head()

🧽 Columnas eliminadas (todo ceros / NaN, umbral=1.0): ['Imp. Neto Gravado IVA 0%', 'IVA 2,5%', 'Imp. Neto Gravado IVA 2,5%', 'IVA 5%', 'Imp. Neto Gravado IVA 5%']
✅ Guardado: ..\..\3_data_final\egresos_totales_final.csv  (columnas: 24 ➜ 19)


,FechaEmision,Año,Mes,Tipo de Comprobante,Nro. Doc. Receptor,DenominacionReceptor,"IVA 10,5%","Imp. Neto Gravado IVA 10,5%",IVA 21%,Imp. Neto Gravado IVA 21%,IVA 27%,Imp. Neto Gravado IVA 27%,Imp. Neto Gravado Total,Imp. Neto No Gravado,Imp. Op. Exentas,Otros Tributos,Total IVA,Imp. Total,TotalVenta
0,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,1704,8115,0,0,8115,0,0,0,1704,9820,9820
1,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,6115,29121,0,0,29121,0,0,0,6115,36402,36402
2,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,1197,5700,0,0,5700,0,0,0,1197,7801,7801
3,2023-01-02,2023,1,1,30657033770,Sin cliente,0,0,8642,41157,0,0,41157,0,0,0,8642,49800,49800
4,2023-01-02,2023,1,1,30657033770,Sin cliente,50435,480335,0,0,0,0,480335,0,0,0,50435,549983,549983


In [9]:
# ==========================================================
# DEFLECTAR INGRESOS_TOTALES A MONEDA CONSTANTE (p.ej. 09/2025)
# ==========================================================

import pandas as pd
import numpy as np
from pathlib import Path

# -------- Parámetros del mes base (lleva todo a precios de...) --------
BASE_ANIO = 2025
BASE_MES  = 10
INCLUIR_MES_ORIGEN = True  # IPC incluye el propio mes (convención habitual)

# --------- Rutas (usando tus constantes) ---------
from utilidades.constantes import ARCHIVO_INGRESOS_FINAL_PATH
INGRESOS_PATH = Path(ARCHIVO_INGRESOS_FINAL_PATH)

# --------- Lectura robusta (sep=';') ----------
df = pd.read_csv(INGRESOS_PATH, sep=";", encoding="utf-8-sig", low_memory=False)

# --------- Normalizar columnas clave (acepta variantes) ----------
# Fecha
if "FechaEmision" in df.columns:
    col_fecha = "FechaEmision"
elif "Fecha de Emisión" in df.columns:
    col_fecha = "Fecha de Emisión"
elif "Fecha de Emision" in df.columns:
    col_fecha = "Fecha de Emision"
else:
    raise ValueError("No encuentro la columna de fecha (FechaEmision / Fecha de Emisión).")

# Tipo comprobante
col_tc = "Tipo de Comprobante" if "Tipo de Comprobante" in df.columns else None
if col_tc is None:
    raise ValueError("No encuentro 'Tipo de Comprobante'.")

# Total nominal
col_total = "Imp. Total" if "Imp. Total" in df.columns else None
if col_total is None:
    raise ValueError("No encuentro 'Imp. Total'.")

# Identificador de receptor para emparejar NCs: usa Nro. Doc. Receptor si existe
col_doc = "Nro. Doc. Receptor" if "Nro. Doc. Receptor" in df.columns else None

# --------- Preparar tipos y columnas auxiliares ----------
df[col_fecha] = pd.to_datetime(df[col_fecha], errors="coerce")
df[col_tc] = df[col_tc].astype(str)

# Volver negativo el nominal para NC (3,8,203) – por si no lo está
df[col_total] = pd.to_numeric(
    df[col_total]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace(r"[^0-9\.-]", "", regex=True)
    .replace("", "0"),
    errors="coerce"
).fillna(0.0)

df.loc[df[col_tc].isin(["3","8","203"]), col_total] *= -1

# Año y Mes de emisión
df["Anio"] = df[col_fecha].dt.year
df["Mes"]  = df[col_fecha].dt.month

# --------- Inflación mensual (enero/2023 → septiembre/2025) ----------
inflaciones_pct = [
    # 2023
    6.0, 6.6, 7.7, 8.4, 7.8, 6.0, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,
    # 2024
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,
    # 2025 (ene–sep)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.9, 2.1, 2.3
]
infl = [x/100 for x in inflaciones_pct]
anios = [2023]*12 + [2024]*12 + [2025]*10
meses = list(range(1,13)) + list(range(1,13)) + list(range(1,11))
tbl = pd.DataFrame({"Anio": anios, "Mes": meses, "inf": infl})

# Índice IPC relativo
if INCLUIR_MES_ORIGEN:
    tbl["IPC"] = (1 + tbl["inf"]).cumprod()
else:
    ipc, acc = [], 1.0
    for r in tbl["inf"]:
        ipc.append(acc)
        acc *= (1 + r)
    tbl["IPC"] = ipc

# IPC del mes base
idx_base = tbl.index[(tbl["Anio"]==BASE_ANIO) & (tbl["Mes"]==BASE_MES)]
if len(idx_base)==0:
    raise ValueError("El mes base no existe en la tabla de inflación.")
IPC_BASE = float(tbl.loc[idx_base[0], "IPC"])

# ==========================================================
# Emparejar Notas de Crédito con ventas (si tenemos Nro. Doc.)
# para usar el mes/año de la venta original como referencia.
# ==========================================================
df["Anio_ajuste"] = df["Anio"]
df["Mes_ajuste"]  = df["Mes"]

if col_doc is not None:
    ventas = df[~df[col_tc].isin(["3","8","203"])].copy()
    ncs    = df[df[col_tc].isin(["3","8","203"])].copy()

    ventas = ventas.sort_values([col_doc, col_fecha])
    ncs    = ncs.sort_values([col_doc, col_fecha])

    # index rápido por documento
    # (si hay docs vacíos, se ignoran en el emparejamiento)
    ventas_valid = ventas.dropna(subset=[col_doc]).copy()

    for idx, nc in ncs.dropna(subset=[col_doc]).iterrows():
        doc    = nc[col_doc]
        fecha  = nc[col_fecha]
        monto  = abs(nc[col_total])

        sub = ventas_valid[(ventas_valid[col_doc]==doc) & (ventas_valid[col_fecha] <= fecha)]
        if sub.empty:
            continue

        sub = sub.copy()
        sub["diff"] = (sub[col_total].abs() - monto).abs()

        # Prioridad: coincidencia exacta por monto; si no, la más cercana
        ex = sub[sub["diff"]==0]
        match = (ex.sort_values(col_fecha).iloc[-1] if not ex.empty
                 else sub.sort_values(["diff", col_fecha]).iloc[0])

        df.at[idx, "Anio_ajuste"] = int(match[col_fecha].year)
        df.at[idx, "Mes_ajuste"]  = int(match[col_fecha].month)

# ==========================================================
# Deflactar (eliminar efecto inflacionario) al mes base
# ==========================================================
df = df.merge(
    tbl[["Anio","Mes","IPC"]].rename(columns={"IPC":"IPC_fila"}),
    left_on=["Anio_ajuste","Mes_ajuste"],
    right_on=["Anio","Mes"],
    how="left",
    suffixes=("_emitido","_tbl")
)

# limpiar columnas auxiliares del merge
df = df.drop(columns=["Anio_tbl","Mes_tbl"], errors="ignore").rename(
    columns={"Anio_emitido":"Anio", "Mes_emitido":"Mes"}
)

# Coeficiente deflactor y valores reales
df["Coef_Deflactor"]   = IPC_BASE / df["IPC_fila"]
df["TotalVenta_Real"]  = (df[col_total] * df["Coef_Deflactor"]).round(0).astype("Int64")
df["TotalVenta_Nominal"] = df[col_total].round(0).astype("Int64")

# ==========================================================
# Guardar
# ==========================================================
SALIDA = INGRESOS_PATH.with_name("ingresos_totales_final_ajustado.csv")
df.to_csv(SALIDA, index=False, sep=";", encoding="utf-8-sig")

print(f"✅ DEFLECTADO a moneda constante de {BASE_MES:02d}/{BASE_ANIO}: {SALIDA}")
print(f"   Filas: {len(df):,} | Columnas: {df.shape[1]}")


✅ DEFLECTADO a moneda constante de 10/2025: ..\..\3_data_final\ingresos_totales_final_ajustado.csv
   Filas: 126,706 | Columnas: 29


In [10]:
# ==========================================================
# DEFLECTAR INGRESOS_TOTALES A MONEDA CONSTANTE (p.ej. 09/2025)
# ==========================================================
import pandas as pd
import numpy as np
from pathlib import Path

# -------- Parámetros del mes base (lleva todo a precios de...) --------
BASE_ANIO = 2025
BASE_MES  = 10
INCLUIR_MES_ORIGEN = True  # IPC incluye el propio mes (convención habitual)

# --------- Rutas (usando tus constantes) ---------
from utilidades.constantes import ARCHIVO_EGRESOS_FINAL_PATH
INGRESOS_PATH = Path(ARCHIVO_EGRESOS_FINAL_PATH)

# --------- Lectura robusta (sep=';') ----------
df = pd.read_csv(INGRESOS_PATH, sep=";", encoding="utf-8-sig", low_memory=False)

# --------- Normalizar columnas clave (acepta variantes) ----------
# Fecha
if "FechaEmision" in df.columns:
    col_fecha = "FechaEmision"
elif "Fecha de Emisión" in df.columns:
    col_fecha = "Fecha de Emisión"
elif "Fecha de Emision" in df.columns:
    col_fecha = "Fecha de Emision"
else:
    raise ValueError("No encuentro la columna de fecha (FechaEmision / Fecha de Emisión).")

# Tipo comprobante
col_tc = "Tipo de Comprobante" if "Tipo de Comprobante" in df.columns else None
if col_tc is None:
    raise ValueError("No encuentro 'Tipo de Comprobante'.")

# Total nominal
col_total = "Imp. Total" if "Imp. Total" in df.columns else None
if col_total is None:
    raise ValueError("No encuentro 'Imp. Total'.")

# Identificador de receptor para emparejar NCs: usa Nro. Doc. Receptor si existe
col_doc = "Nro. Doc. Receptor" if "Nro. Doc. Receptor" in df.columns else None

# --------- Preparar tipos y columnas auxiliares ----------
df[col_fecha] = pd.to_datetime(df[col_fecha], errors="coerce")
df[col_tc] = df[col_tc].astype(str)

# Volver negativo el nominal para NC (3,8,203) – por si no lo está
df[col_total] = pd.to_numeric(
    df[col_total]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace(r"[^0-9\.-]", "", regex=True)
    .replace("", "0"),
    errors="coerce"
).fillna(0.0)

df.loc[df[col_tc].isin(["3","8","203"]), col_total] *= -1

# Año y Mes de emisión
df["Anio"] = df[col_fecha].dt.year
df["Mes"]  = df[col_fecha].dt.month

# --------- Inflación mensual (enero/2023 → septiembre/2025) ----------
inflaciones_pct = [
    # 2023
    6.0, 6.6, 7.7, 8.4, 7.8, 6.0, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,
    # 2024
    20.6, 13.2, 11.0, 8.8, 4.2, 4.6, 4.0, 4.2, 3.5, 2.7, 2.4, 2.7,
    # 2025 (ene–sep)
    2.2, 2.4, 3.7, 2.8, 1.5, 1.6, 1.9, 1.9, 2.1, 2.3
]
infl = [x/100 for x in inflaciones_pct]
anios = [2023]*12 + [2024]*12 + [2025]*10
meses = list(range(1,13)) + list(range(1,13)) + list(range(1,11))
tbl = pd.DataFrame({"Anio": anios, "Mes": meses, "inf": infl})

# Índice IPC relativo
if INCLUIR_MES_ORIGEN:
    tbl["IPC"] = (1 + tbl["inf"]).cumprod()
else:
    ipc, acc = [], 1.0
    for r in tbl["inf"]:
        ipc.append(acc)
        acc *= (1 + r)
    tbl["IPC"] = ipc

# IPC del mes base
idx_base = tbl.index[(tbl["Anio"]==BASE_ANIO) & (tbl["Mes"]==BASE_MES)]
if len(idx_base)==0:
    raise ValueError("El mes base no existe en la tabla de inflación.")
IPC_BASE = float(tbl.loc[idx_base[0], "IPC"])

# ==========================================================
# Emparejar Notas de Crédito con ventas (si tenemos Nro. Doc.)
# para usar el mes/año de la venta original como referencia.
# ==========================================================
df["Anio_ajuste"] = df["Anio"]
df["Mes_ajuste"]  = df["Mes"]

if col_doc is not None:
    ventas = df[~df[col_tc].isin(["3","8","203"])].copy()
    ncs    = df[df[col_tc].isin(["3","8","203"])].copy()

    ventas = ventas.sort_values([col_doc, col_fecha])
    ncs    = ncs.sort_values([col_doc, col_fecha])

    # index rápido por documento
    # (si hay docs vacíos, se ignoran en el emparejamiento)
    ventas_valid = ventas.dropna(subset=[col_doc]).copy()

    for idx, nc in ncs.dropna(subset=[col_doc]).iterrows():
        doc    = nc[col_doc]
        fecha  = nc[col_fecha]
        monto  = abs(nc[col_total])

        sub = ventas_valid[(ventas_valid[col_doc]==doc) & (ventas_valid[col_fecha] <= fecha)]
        if sub.empty:
            continue

        sub = sub.copy()
        sub["diff"] = (sub[col_total].abs() - monto).abs()

        # Prioridad: coincidencia exacta por monto; si no, la más cercana
        ex = sub[sub["diff"]==0]
        match = (ex.sort_values(col_fecha).iloc[-1] if not ex.empty
                 else sub.sort_values(["diff", col_fecha]).iloc[0])

        df.at[idx, "Anio_ajuste"] = int(match[col_fecha].year)
        df.at[idx, "Mes_ajuste"]  = int(match[col_fecha].month)

# ==========================================================
# Deflactar (eliminar efecto inflacionario) al mes base
# ==========================================================
df = df.merge(
    tbl[["Anio","Mes","IPC"]].rename(columns={"IPC":"IPC_fila"}),
    left_on=["Anio_ajuste","Mes_ajuste"],
    right_on=["Anio","Mes"],
    how="left",
    suffixes=("_emitido","_tbl")
)

# limpiar columnas auxiliares del merge
df = df.drop(columns=["Anio_tbl","Mes_tbl"], errors="ignore").rename(
    columns={"Anio_emitido":"Anio", "Mes_emitido":"Mes"}
)

# Coeficiente deflactor y valores reales
df["Coef_Deflactor"]   = IPC_BASE / df["IPC_fila"]
df["TotalVenta_Real"]  = (df[col_total] * df["Coef_Deflactor"]).round(0).astype("Int64")
df["TotalVenta_Nominal"] = df[col_total].round(0).astype("Int64")

# ==========================================================
# Guardar
# ==========================================================
SALIDA = INGRESOS_PATH.with_name("egresos_totales_final_ajustado.csv")
df.to_csv(SALIDA, index=False, sep=";", encoding="utf-8-sig")

print(f"✅ DEFLECTADO a moneda constante de {BASE_MES:02d}/{BASE_ANIO}: {SALIDA}")
print(f"   Filas: {len(df):,} | Columnas: {df.shape[1]}")

✅ DEFLECTADO a moneda constante de 10/2025: ..\..\3_data_final\egresos_totales_final_ajustado.csv
   Filas: 7,852 | Columnas: 31
